In [ ]:
# ✅ LangGraph 기반으로 리팩토링된 agent.py

import os
from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

from llm_tools.retriever import RAG_tool
from llm_tools.get_weather import get_weather_by_location_and_date

load_dotenv()

# ✅ 1. Agent 및 Tool 설정 (기존과 동일)
cur_date = datetime.now()

tools = [RAG_tool, get_weather_by_location_and_date]

agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"""
당신은 문화 유산 탐사대입니다.
현재 날짜는 {cur_date}입니다.

[Guidelines]
- 대한민국의 문화유산에 대한 정보는 RAG_tool 사용
- 날씨 정보는 get_weather_by_location_and_date 사용
"""),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)
    ]
)

agent = create_tool_calling_agent(
    llm=ChatOpenAI(model_name="gpt-4.1"),
    tools=tools,
    prompt=agent_prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools)

# ✅ 2. LangGraph용 상태 정의
class GraphState(TypedDict):
    query: str
    response: Optional[str]

# ✅ 3. 노드 정의 (각 단계를 함수로 분리)
def parse_node(state: GraphState) -> GraphState:
    return {"query": state["query"]}

def agent_node(state: GraphState) -> GraphState:
    result = agent_executor.invoke({"query": state["query"]})
    return {"query": state["query"], "response": result["output"]}

def respond_node(state: GraphState) -> GraphState:
    print(f"\n🧠 응답: {state['response']}")
    return state

# ✅ 4. LangGraph 그래프 구성
graph = StateGraph(GraphState)
graph.add_node("parse", parse_node)
graph.add_node("run_agent", agent_node)
graph.add_node("respond", respond_node)

graph.set_entry_point("parse")
graph.add_edge("parse", "run_agent")
graph.add_edge("run_agent", "respond")
graph.add_edge("respond", END)

app = graph.compile()

# ✅ 5. 실행 루프 (간단한 입력 반복)
while True:
    query = input("\n\n >>> 쿼리를 입력하세요: ")
    if query == "!quit":
        break
    app.invoke({"query": query})


🧠 응답: 안녕하세요! 저는 대한민국의 문화유산을 탐사하고 안내해드리는 “문화 유산 탐사대”입니다.  
한국의 다양한 문화재, 사적, 기념물, 전통문화 등에 대해 궁금한 점이 있으면 언제든 질문해 주세요.  
현장 답사, 역사 배경, 가치 등 상세한 설명과 최신 정보를 제공해드릴 수 있습니다.  
오늘 특별히 궁금한 문화유산이나, 알고 싶은 지역이 있으신가요?


c:\Users\jhwoo\Desktop\SKN_ws\project\project3_ws\SKN13-3rd-5Team\llm_tools\retriever.py:31: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(
c:\Users\jhwoo\Desktop\SKN_ws\project\project3_ws\SKN13-3rd-5Team\llm_tools\retriever.py:39: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



🧠 응답: 경복궁은 서울특별시 종로구에 위치해 있습니다. 조선시대에 건립된 대표적인 왕궁으로, 한양(현재의 서울)으로 수도를 옮긴 태조 4년(1395)에 처음으로 세워졌습니다. 종로구 세종로 일대에 있으며, 지금도 서울의 중심 관광지이자 역사적 명소로 많은 사람들이 찾고 있습니다.

정확한 주소는 "서울특별시 종로구 세종로 1-1"입니다.

🧠 응답: 경복궁은 서울특별시 종로구에 위치해 있습니다. 한양(현재의 서울)으로 수도를 옮긴 후 1395년에 처음으로 세운 조선시대의 중심 궁궐로, 서울 도심 한복판에 자리잡고 있습니다.

정확한 주소는 "서울특별시 종로구 사직로 161"입니다.

🧠 응답: 죄송합니다. 현재 기상청 데이터로 내일(2025-06-28) 경복궁이 위치한 서울 종로구의 날씨 정보를 불러오지 못했습니다.

다른 궁금하신 점이나 서울 지역 일반적인 기상 경향, 또는 경복궁에 대한 문화유산 정보가 필요하시면 알려드릴 수 있습니다!

🧠 응답: 경복궁은 서울특별시 종로구에 위치해 있습니다. "경복궁"이라는 이름으로는 날씨 정보를 찾을 수 없으니, 대신 "서울"로 확인해드릴까요? 확인이 필요하시면 말씀해 주세요!

🧠 응답: 서울의 주요 문화유산 몇 가지를 소개해드립니다.

1. 백악산(북악산)
- 조선왕조가 한양(현재의 서울)을 도성으로 삼은 데 결정적 역할을 한 산입니다. 안정감과 기운을 상징하는 풍수적 의미를 가지며, 한양 도성의 북쪽 주산으로 조선시대 경관과 도성 배치의 원리를 보여줍니다. 경복궁의 후원 역할도 했던 자연·문화 복합유산입니다.

2. 경복궁
- 조선의 첫 번째 정궁이자 대한민국을 대표하는 궁궐로, 왕실의 정치와 문화의 중심지였습니다. 조선 건축과 조경, 미의식이 집약된 상징적인 장소입니다.

3. 창덕궁
- 자연과 조화를 이루는 후원이 유명하며, 유네스코 세계문화유산에 등재되어 있습니다. 조선왕조 궁궐 건축의 백미로 꼽힙니다.

4. 종묘
- 조선 왕조 역대 왕과 왕비의 신위를 모시는 곳으로, 유네스코 세계유산입니다. 제례와 음